In [103]:
import numpy as np
#matplotlib inline
import matplotlib.pyplot as plt
#Import math module
#import math #Don't use with numpy. They have different definitions for the same things.
#Import LMfit to minimize chi^2.
import lmfit
from lmfit import Minimizer, Parameters, report_fit
#from lmfit import minimize, Parameters

#Constants
pi = 3.141592654
deg2rad = pi/180.0
GeV2fm = 1./0.0389             #Convert Q^2 units from GeV^2 to fm^-2.
hbar = 6.582*np.power(10.0,-16.0)   #hbar in [eV*s].
alpha = 0.0072973525664        #Fine structure constant.
C = 299792458.0                #Speed of light [m/s].
e = 1.60217662E-19             #Electron charge [C].
e2_nuclear = 1.4399643929e-3   #Electron charge squared in nuclear units [GeV * fm].
MtHe3 = 3.0160293*0.9315       #Mass of He3 in GeV.
muHe3 = -2.1275*(3.0/2.0)      #Magnetic moment of 3He.
Z = 2                          #Atomic number of 3He.
A = 3                          #Number of nucleons.
gamma = 0.8*np.power(2.0/3.0,0.5);  #Gaussian width [fm] from Amroun gamma*sqrt(3/2) = 0.8 fm.

#Flags and Variables that are often modified.
ngaus = 12
print_data = 0
print_XS_debug = 0
R = [0.3,0.7,0.9,1.1,1.5,1.6,2.2,2.7,3.3,4.2,4.3,4.8]


#f = open('3He_Data.txt', 'r')

E0,theta,sigexp,uncertainty,dataset = np.loadtxt('3He_Data.txt', skiprows=2, unpack=True)

#data = np.genfromtxt(f, delimiter=' ')
#delete(data,0,0) # Erases the first row (i.e. the header)

#E0 = data[:,0]
#theta = data[:,1]
#sigexp = data[:,2]
#uncertainty = data[:,3]

if print_data == 1:
    print(E0)
    print(theta)
    print(sigexp)
    print(uncertainty)
    print(dataset)


#f.close()

#print(np.pi)

#Define charge form factor function.
def Fch(Qich):
    return

#Define cross section function. Takes in E0 and theta along with free parameters Qich and Qim.
def XS(E0,theta,Q0ch,Q1ch,Q2ch,Q3ch,Q4ch,Q5ch,Q6ch,Q7ch,Q8ch,Q9ch,Q10ch,Q11ch,Q0m,Q1m,Q2m,Q3m,Q4m,Q5m,Q6m,Q7m,Q8m,Q9m,Q10m,Q11m):
    
    pars = [Q0ch,Q1ch,Q2ch,Q3ch,Q4ch,Q5ch,Q6ch,Q7ch,Q8ch,Q9ch,Q10ch,Q11ch,Q0m,Q1m,Q2m,Q3m,Q4m,Q5m,Q6m,Q7m,Q8m,Q9m,Q10m,Q11m]
    
    #Define variables needed to get XS.
    Ef = E0/(1.0+2.0*E0*np.power(np.sin(theta*deg2rad/2.0),2.0)/MtHe3)     #Scattered electron energy (GeV).
    Q2 = 4.0*E0*Ef*np.power(np.sin(theta*deg2rad/2.0),2.0) * GeV2fm        #Squared four-momentum (fm^-2).
    Q2eff = np.power( np.power(Q2,0.5) * (1.0+(1.5*Z*alpha)/(E0*np.power(GeV2fm,0.5)*1.12*np.power(A,1.0/3.0))) ,2.0)  #Effective Q^2 (fm^-2).
    W = E0 - Ef                                                         #Energy transfer (GeV).
    q2_3 = abs(  np.power(W,2.0)*GeV2fm - Q2eff  )                      #Three-momentum squared (fm^-2).
    eta = 1.0 + Q2eff/(4.0*np.power(MtHe3,2.0)*GeV2fm)                  #Constant eta from Rosenbluth equation in Amroun 1994.
    #Mott XS (fm^-2).
    mottxs = (  (np.power(Z,2.)*(Ef/E0)) * (np.power(alpha,2.0)/(4.0*np.power(E0,2.0)*np.power(np.sin(theta*deg2rad/2.0),4.0)))*np.power(np.cos(theta*deg2rad/2.0),2.0)  ) * 1.0/25.7
    
    #Loop to sum the charge form factor.
    #par = [0.0996392,0.214304,0.0199385,0.195676,0.0785533,0.167223,0.126926,0.0549379,0.0401401,0.0100803,0.0007217,4.98962e-12,0.159649,0.0316168,0.277843,0.0364955,0.0329718,0.233469,0.117059,0.0581085,0.0485212,1.77602e-12,0.0240927,8.94934e-12]
    #par = [Q0ch,Q1ch,Q2ch,Q3ch,Q4ch,Q5ch,Q6ch,Q7ch,Q8ch,Q9ch,Q10ch,Q11ch,Q0m,Q1m,Q2m,Q3m,Q4m,Q5m,Q6m,Q7m,Q8m,Q9m,Q10m,Q11m]
    Fch = 0
    for i in range(ngaus):
        sumFch = (pars[i]/(1.0+2.0*np.power(R[i],2.0)/np.power(gamma,2.0))) * ( np.cos(np.power(Q2eff,0.5)*R[i]) + (2.0*np.power(R[i],2.0)/np.power(gamma,2.0)) * (np.sin(np.power(Q2eff,0.5)*R[i])/(np.power(Q2eff,0.5)*R[i])) )
        Fch = Fch +sumFch
    Fch =  Fch * np.exp(-0.25*Q2eff*np.power(gamma,2.0))
        
    #Loop to sum the magnetic form factor.
    Fm = 0
    for i in range(ngaus):
        sumFm = (pars[i+ngaus]/(1.0+2.0*np.power(R[i],2.0)/np.power(gamma,2.0))) * ( np.cos(np.power(Q2eff,0.5)*R[i]) + (2.0*np.power(R[i],2.0)/np.power(gamma,2.0)) * (np.sin(np.power(Q2eff,0.5)*R[i])/(np.power(Q2eff,0.5)*R[i])) )
        Fm = Fm +sumFm
    Fm =  Fm * np.exp(-0.25*Q2eff*np.power(gamma,2.0))
    
    #Calculate full cross section.
    XS = mottxs * (1./eta) * ( (Q2eff/q2_3)*np.power(Fch,2.) + (np.power(muHe3,2.0)*Q2eff/(2*np.power(MtHe3,2)*GeV2fm))*(0.5*Q2eff/q2_3 + np.power(np.tan(theta*deg2rad/2),2))*np.power(Fm,2.) )
    
    if print_XS_debug == 1:
        print('Ef = ',Ef)
        print('Q^2 = ',Q2)
        print('Q^2 Effective = ',Q2eff)
        print('W = ',W)
        print('Three-momentum Squared = ',q2_3)
        print('eta = ',eta)
        print('Mott XS = ',mottxs)
        print('Fch = ',Fch)
        print('Fm = ',Fm)
        print('XS = ',XS)
    
    return XS

#XS(E0[0],theta[0])

#def chi2(params, Q0ch,Q1ch,Q2ch,Q3ch,Q4ch,Q5ch,Q6ch,Q7ch,Q8ch,Q9ch,Q10ch,Q11ch,Q0m,Q1m,Q2m,Q3m,Q4m,Q5m,Q6m,Q7m,Q8m,Q9m,Q10m,Q11m):
# def chi(params, sigexp, uncertainty, E0, theta):
    
#     E0_temp = E0
#     theta_temp = theta
    
#     Q0ch = params['Q0ch']
#     Q1ch = params['Q1ch']
#     Q2ch = params['Q2ch']
#     Q3ch = params['Q3ch']    
#     Q4ch = params['Q4ch']
#     Q5ch = params['Q5ch']
#     Q6ch = params['Q6ch']
#     Q7ch = params['Q7ch'] 
#     Q8ch = params['Q8ch']
#     Q9ch = params['Q9ch']
#     Q10ch = params['Q10ch']
#     Q11ch = params['Q11ch'] 
#     Q0m = params['Q0m']
#     Q1m = params['Q1m']
#     Q2m = params['Q2m']
#     Q3m = params['Q3m']    
#     Q4m = params['Q4m']
#     Q5m = params['Q5m']
#     Q6m = params['Q6m']
#     Q7m = params['Q7m'] 
#     Q8m = params['Q8m']
#     Q9m = params['Q9m']
#     Q10m = params['Q10m']
#     Q11m = params['Q11m'] 
    
#     delta = (sigexp-XS(E0,theta,Q0ch,Q1ch,Q2ch,Q3ch,Q4ch,Q5ch,Q6ch,Q7ch,Q8ch,Q9ch,Q10ch,Q11ch,Q0m,Q1m,Q2m,Q3m,Q4m,Q5m,Q6m,Q7m,Q8m,Q9m,Q10m,Q11m))/uncertainty
#     #delta = (sigexp-XS(E0_temp,theta_temp,Q0ch,Q1ch,Q2ch,Q3ch,Q4ch,Q5ch,Q6ch,Q7ch,Q8ch,Q9ch,Q10ch,Q11ch,Q0m,Q1m,Q2m,Q3m,Q4m,Q5m,Q6m,Q7m,Q8m,Q9m,Q10m,Q11m))/uncertainty
#     #delta = (sigexp-XS(E0,theta,Q0ch,Q1ch,Q2ch,Q3ch,Q4ch,Q5ch,Q6ch,Q7ch,Q8ch,Q9ch,Q10ch,Q11ch,Q0m,Q1m,Q2m,Q3m,Q4m,Q5m,Q6m,Q7m,Q8m,Q9m,Q10m,Q11m))/uncertainty
#     #delta = (sigexp-Q0ch)/uncertainty
#     #chi2 = delta ** 2
    
#     return chi

# params = Parameters()
# params.add('Q0ch', value=0.0996392, min=0)
# params.add('Q1ch', value=0.214304, min=0)
# params.add('Q2ch', value=0.0199385, min=0)
# params.add('Q3ch', value=0.195676, min=0)
# params.add('Q4ch', value=0.0785533, min=0)
# params.add('Q5ch', value=0.167223, min=0)
# params.add('Q6ch', value=0.126926, min=0)
# params.add('Q7ch', value=0.0549379, min=0)
# params.add('Q8ch', value=0.0401401, min=0)
# params.add('Q9ch', value=0.0100803, min=0)
# params.add('Q10ch', value=0.0007217, min=0)
# params.add('Q11ch', value=4.98962e-12, min=0)
# params.add('Q0m', value=0.159649, min=0)
# params.add('Q1m', value=0.0316168, min=0)
# params.add('Q2m', value=0.277843, min=0)
# params.add('Q3m', value=0.0364955, min=0)
# params.add('Q4m', value=0.0329718, min=0)
# params.add('Q5m', value=0.233469, min=0)
# params.add('Q6m', value=0.117059, min=0)
# params.add('Q7m', value=0.0581085, min=0)
# params.add('Q8m', value=0.0485212, min=0)
# params.add('Q9m', value=1.77602e-12, min=0)
# params.add('Q10m', value=0.0240927, min=0)
# params.add('Q11m', value=8.94934e-12, min=0)

# minner = Minimizer(chi2, params, fcn_args=(sigexp, uncertainty, E0, theta))
# result = minner.minimize()
# final = sigexp + result.residual
# report_fit(result)

In [104]:
from lmfit import Model

In [105]:
model=Model(XS, independent_vars=['E0','theta'])
model.set_param_hint('Q0ch', value=0.0996392, min=0, max=1.0)
model.set_param_hint('Q1ch', value=0.214304, min=0, max=1.0)
model.set_param_hint('Q2ch', value=0.0199385, min=0, max=1.0)
model.set_param_hint('Q3ch', value=0.195676, min=0, max=1.0)
model.set_param_hint('Q4ch', value=0.0785533, min=0, max=1.0)
model.set_param_hint('Q5ch', value=0.167223, min=0, max=1.0)
model.set_param_hint('Q6ch', value=0.126926, min=0, max=1.0)
model.set_param_hint('Q7ch', value=0.0549379, min=0, max=1.0)
model.set_param_hint('Q8ch', value=0.0401401, min=0, max=1.0)
model.set_param_hint('Q9ch', value=0.0100803, min=0, max=1.0)
model.set_param_hint('Q10ch', value=0.0007217, min=0, max=1.0)
model.set_param_hint('Q11ch', value=4.98962e-12, min=0, max=1.0)
model.set_param_hint('Q0m', value=0.159649, min=0, max=1.0)
model.set_param_hint('Q1m', value=0.0316168, min=0, max=1.0)
model.set_param_hint('Q2m', value=0.277843, min=0, max=1.0)
model.set_param_hint('Q3m', value=0.0364955, min=0, max=1.0)
model.set_param_hint('Q4m', value=0.0329718, min=0, max=1.0)
model.set_param_hint('Q5m', value=0.233469, min=0, max=1.0)
model.set_param_hint('Q6m', value=0.117059, min=0, max=1.0)
model.set_param_hint('Q7m', value=0.0581085, min=0, max=1.0)
model.set_param_hint('Q8m', value=0.0485212, min=0, max=1.0)
model.set_param_hint('Q9m', value=1.77602e-12, min=0, max=1.0)
model.set_param_hint('Q10m', value=0.0240927, min=0, max=1.0)
model.set_param_hint('Q11m', value=8.94934e-12, min=0, max=1.0)

In [106]:
result=model.fit(sigexp,E0=E0,theta=theta,Q0ch=0.0996392,Q1ch=0.214304,Q2ch=0.0199385,Q3ch=0.195676,Q4ch=0.0785533,Q5ch=0.167223,Q6ch=0.126926,Q7ch=0.0549379,Q8ch=0.0401401,Q9ch=0.0100803,Q10ch=0.0007217,Q11ch=4.98962e-12,Q0m=0.159649,Q1m=0.0316168,Q2m=0.277843,Q3m=0.0364955,Q4m=0.0329718,Q5m=0.233469,Q6m=0.117059,Q7m=0.0581085,Q8m=0.0485212,Q9m=1.77602e-12,Q10m=0.0240927,Q11m=8.94934e-12,weights=uncertainty)

In [107]:
print(result.fit_report())

[[Model]]
    Model(XS)
[[Fit Statistics]]
    # fitting method   = leastsq
    # function evals   = 1841
    # data points      = 259
    # variables        = 24
    chi-square         = 3.2101e-15
    reduced chi-square = 1.3660e-17
    Akaike info crit   = -10034.6905
    Bayesian info crit = -9949.32665
##  Warning: uncertainties could not be estimated:
    Q9m:    at initial value
    Q9m:    at boundary
    Q11m:   at initial value
    Q11m:   at boundary
[[Variables]]
    Q0ch:   0.09427293 +/-        nan (nan%) (init = 0.0996392)
    Q1ch:   0.21044551 +/-        nan (nan%) (init = 0.214304)
    Q2ch:   0.01722478 +/-        nan (nan%) (init = 0.0199385)
    Q3ch:   0.19376051 +/-        nan (nan%) (init = 0.195676)
    Q4ch:   0.07882869 +/-        nan (nan%) (init = 0.0785533)
    Q5ch:   0.16805421 +/- 27755.5229 (16515814.82%) (init = 0.167223)
    Q6ch:   0.13057474 +/- 26529.3242 (20317347.29%) (init = 0.126926)
    Q7ch:   0.05954991 +/-        nan (nan%) (init = 0.05493

In [101]:
Qich = [result.best_values['Q0ch'],result.best_values['Q1ch'],result.best_values['Q2ch'],result.best_values['Q3ch'],result.best_values['Q4ch'],result.best_values['Q5ch'],result.best_values['Q6ch'],result.best_values['Q7ch'],result.best_values['Q8ch'],result.best_values['Q9ch'],result.best_values['Q10ch'],result.best_values['Q11ch']]
print(Qich)
Qim = [result.best_values['Q0m'],result.best_values['Q1m'],result.best_values['Q2m'],result.best_values['Q3m'],result.best_values['Q4m'],result.best_values['Q5m'],result.best_values['Q6m'],result.best_values['Q7m'],result.best_values['Q8m'],result.best_values['Q9m'],result.best_values['Q10m'],result.best_values['Q11m']]
print(Qim)

[0.09427293488222527, 0.2104455110388016, 0.017224781763787667, 0.19376050942078243, 0.07882868604741944, 0.16805421453373698, 0.13057474409558806, 0.05954990775902025, 0.04352734617162629, 0.010513342119978675, 0.002224987379768284, 0.008964663488188751]
[2.6472179803060936e-05, 6.71800203266848e-06, 2.899835477038204e-05, 5.601867549565043e-05, 0.000665781938388954, 0.00025934478657607674, 0.0003651289396051638, 0.0005443544373837095, 2.1404084041831695e-05, 2.0317081350640365e-14, 0.00010607594370587448, 2.0453196358793946e-09]
